In [74]:
# binaural masking level difference
# use headphones, first set VOLUME DOWN!!! DO NOT HURT YOUR EARS!!!
# the slider "level" sets the level of the tone
# the slider "phase diff" sets the phase difference between the tone in the left and right ear
# set the slider "phase diff" to zero and adjust the tone level until the tone is masked by noise
# then shift the slider "phase diff" to 180 deg. and play the tone again
# if the tone was just masked for 0 phase diff, it should be unmasked if the phase diff is 180 deg.


import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import IPython.display as ipd   
import ipywidgets as widgets


Tdur = 3 # noise duration (seconds)
fs = 44100 # sample rate (Hz)
dt = 1/fs  # time step
tx = np.arange(0,Tdur,dt)  # time axis
freq = 0.5e3 # frequency of the tone
nsamp = len(tx)
amp = 0.01 # amplitude of the noise
noise = amp*np.random.normal(0,1,nsamp) # broad band noise

# make a low pass filter up to 10 kHz to make the noise less annoying 
cutoff = 5e3 # cutoff frequency
normalCutoff = cutoff / (fs/2)
order = 15
bLP, aLP = signal.butter(order, normalCutoff, btype='low')

fnoise = signal.lfilter(bLP, aLP, noise)


#ipd.display(ipd.Audio(hpitch, rate=fs))


#fig, ax = plt.subplots(figsize=(13, 6))
#ax.set_xlabel('time')
#ax.set_ylabel('amplitude')
     
#ax.plot(tx, amp*noise, lw=3, c='r')
#tone = gen_tone(amp,1e3,tx,0)
#ax.plot(tx, tone, lw=3, c='r')

def gen_tone(amp,freq,tx,phase):
    '''generate tone '''
    
    tone = amp*np.sin(2*np.pi*freq*tx + phase)
    # make a ramp (fade in fade out)
    Rdur = 50e-3  # ramp duration
    x = np.arange(0,Rdur,tx[1]-tx[0])
    x = np.pi*x/Rdur
    rampUp = (1 + np.cos(x + np.pi))/2; # raised cosine onset
    rampDown = np.flip(rampUp)
    
    wholeramp = np.concatenate((rampUp, np.ones(len(tone)-2*len(x)), rampDown))
    
    tone = wholeramp*tone

    
    return tone

def update_signal(ipd1,Lt):
    
    ampt = 10**(Lt/20)
    
    tone_l = gen_tone(ampt,freq,tx,0)
    tone_r = gen_tone(ampt,freq,tx,ipd1/180*np.pi)
    signal_l = tone_l + fnoise
    signal_r = tone_r + fnoise
    signal = [signal_l, signal_r]  # pitch sensation
    
    #fig, ax = plt.subplots(figsize=(13, 6))
    #ax.set_xlabel('time (seconds)')
    #ax.set_ylabel('Amplitude')
     
    #ax.plot(tx,signal, lw=3, c='r')
    
    ipd.display(ipd.Audio(signal, rate=fs,normalize=False))
    
    

c_slide = widgets.IntSlider(min=0,max=180,step=10,description='phase diff')
s_slide = widgets.IntSlider(min=-80,max=-50,step=1,description='level')

widgets.interact(update_signal, ipd1=c_slide, Lt=s_slide)






       


interactive(children=(IntSlider(value=0, description='phase diff', max=180, step=10), IntSlider(value=-50, des…

<function __main__.update_signal(ipd1, Lt)>